In [70]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
# from sklearn.linear_model import HuberRegressor
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.metrics import mean_absolute_error

# # Load the datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Define the target and features
# X_train = train_df.drop(columns=['id', 'Age'])
# y_train = train_df['Age']
# X_test = test_df.drop(columns=['id'])

# # Categorical and numerical columns
# categorical_cols = ['Sex']
# numerical_cols = X_train.columns.difference(categorical_cols)

# # Preprocessing pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', Pipeline([
#             ('imputer', SimpleImputer(strategy='mean')),
#             ('poly', PolynomialFeatures(degree=4, include_bias=False, interaction_only=True)),
#             ('scaler', StandardScaler())
#         ]), numerical_cols),
#         ('cat', Pipeline([
#             ('imputer', SimpleImputer(strategy='most_frequent')),
#             ('encoder', OneHotEncoder(drop='first'))  # Use OneHotEncoder for categorical columns
#         ]), categorical_cols)
#     ])

# # Create the full pipeline with HuberRegressor
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('model', HuberRegressor())
# ])

# # Split the data into training and validation sets
# X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# # Train the model
# pipeline.fit(X_train_split, y_train_split)

# # Predict on the validation set
# y_val_pred = pipeline.predict(X_val)

# # Calculate MAE on the validation set
# mae = mean_absolute_error(y_val, y_val_pred)
# print(f'Mean Absolute Error (MAE): {mae:.4f}')

# # Predict on the test set
# test_predictions = pipeline.predict(X_test)

# # Prepare the submission file
# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'Age': test_predictions
# })

# # Save the submission file
# submission.to_csv('submission.csv', index=False)

# print("Submission file created: submission.csv")


In [71]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, FunctionTransformer, KBinsDiscretizer
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import StackingRegressor
# from sklearn.metrics import mean_absolute_error

# # Load the datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Define the feature engineering function
# def feature_funk(df):
#     df['Surface Area'] = 2 * (df['Length'] * df['Diameter'] + df['Length'] * df['Height'] + df['Diameter'] * df['Height'])
#     df['Volume'] = df['Length'] * df['Diameter'] * df['Height']
#     df['Log Weight'] = df['Weight'].apply(lambda x: np.log(x + 1) if x > 0 else np.nan)
#     df['Length Bins'] = pd.qcut(df['Length'], q=4, labels=False)
    
#     # Add interaction features
#     df['Length_Diameter'] = df['Length'] * df['Diameter']
#     df['Length_Height'] = df['Length'] * df['Height']
#     df['Diameter_Height'] = df['Diameter'] * df['Height']
    
#     sex = {'I': 1, 'F': 3, 'M': 2}
#     df['Sex'] = df['Sex'].replace(sex)
#     df = df[df['Sex'].isin([1, 2, 3])]
#     df['Sex'] = pd.to_numeric(df['Sex'])
#     return df

# # Apply the feature engineering function to both datasets
# train_df = feature_funk(train_df)
# test_df = feature_funk(test_df)

# # Define features and target
# X = train_df.drop(columns=['id', 'Age'])
# y = train_df['Age']
# X_test = test_df.drop(columns=['id'])

# # Define categorical and numerical columns
# categorical_cols = ['Sex']
# numerical_cols = [col for col in X.columns if col not in categorical_cols]

# # Define specific columns for discretization
# discretized_cols = ['Length', 'Weight']

# # Updated numerical preprocessing pipeline with discretization for specific columns
# num_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('discretizer', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')),
#     ('poly', PolynomialFeatures(degree=3, include_bias=False)),
#     ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
#     ('scaler', StandardScaler())
# ])

# # Updated categorical preprocessing pipeline
# cat_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
# ])

# # Combine all features
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', num_pipeline, [col for col in numerical_cols if col in discretized_cols]),
#         ('cat', cat_pipeline, categorical_cols),
#     ])

# # Define base models
# base_models = [
#     ('lin_reg', LinearRegression())
# ]

# # Define the meta-model
# meta_model = LinearRegression()

# # Stacking Regressor
# stacking_regressor = StackingRegressor(
#     estimators=base_models,
#     final_estimator=meta_model,
#     cv=5,
#     n_jobs=-1
# )

# # Full pipeline with stacking
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('stacking', stacking_regressor)
# ])

# # Hyperparameter tuning using GridSearchCV
# param_grid = {
#     'preprocessor__num__discretizer__n_bins': [5, 10],
#     'preprocessor__num__poly__degree': [2, 3]
# }

# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
# grid_search.fit(X, y)

# # Best model evaluation
# best_model = grid_search.best_estimator_
# cv_scores = cross_val_score(best_model, X, y, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring='neg_mean_absolute_error')
# mean_cv_mae = -np.mean(cv_scores)
# print(f'Best Model Cross-validated Mean Absolute Error (MAE): {mean_cv_mae:.4f}')

# # Fit the best model on the entire training data
# best_model.fit(X, y)

# # Predict on the test set
# test_predictions = best_model.predict(X_test)

# # Ensure test_predictions length matches test_df
# if len(test_predictions) != len(test_df):
#     raise ValueError("The number of test predictions does not match the number of test data rows.")

# # Create submission file
# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'Age': test_predictions
# })
# submission.to_csv('submission.csv', index=False)

# print("Submission file created: submission.csv")
# y

In [72]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, FunctionTransformer, KBinsDiscretizer
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import StackingRegressor
# from sklearn.metrics import mean_absolute_error

# # Load the datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Define the feature engineering function
# def feature_funk(df):
#     df['Surface Area'] = 2 * (df['Length'] * df['Diameter'] + df['Length'] * df['Height'] + df['Diameter'] * df['Height'])
#     df['Volume'] = df['Length'] * df['Diameter'] * df['Height']
#     df['Log Weight'] = df['Weight'].apply(lambda x: np.log(x + 1) if x > 0 else np.nan)
#     df['Length Bins'] = pd.qcut(df['Length'], q=4, labels=False)
    
#     # Interaction features
#     df['Length_Diameter'] = df['Length'] * df['Diameter']
#     df['Length_Height'] = df['Length'] * df['Height']
#     df['Diameter_Height'] = df['Diameter'] * df['Height']
#     df['Length_Log Weight'] = df['Length'] * df['Log Weight']
    
#     sex = {'I': 1, 'F': 3, 'M': 2}
#     df['Sex'] = df['Sex'].replace(sex)
#     df = df[df['Sex'].isin([1, 2, 3])]
#     df['Sex'] = pd.to_numeric(df['Sex'])
#     return df

# # Apply the feature engineering function to both datasets
# train_df = feature_funk(train_df)
# test_df = feature_funk(test_df)

# # Define features and target
# X = train_df.drop(columns=['id', 'Age'])
# y = train_df['Age']
# X_test = test_df.drop(columns=['id'])

# # Define categorical and numerical columns
# categorical_cols = ['Sex']
# numerical_cols = [col for col in X.columns if col not in categorical_cols]

# # Define specific columns for discretization
# discretized_cols = ['Length', 'Weight', 'Volume']

# # Updated numerical preprocessing pipeline with discretization for specific columns
# num_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('discretizer', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')),
#     ('poly', PolynomialFeatures(degree=3, include_bias=False)),
#     ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
#     ('scaler', StandardScaler())
# ])

# # Updated categorical preprocessing pipeline
# cat_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
# ])

# # Combine all features
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', num_pipeline, [col for col in numerical_cols if col in discretized_cols]),
#         ('cat', cat_pipeline, categorical_cols),
#     ])

# # Define base models for stacking
# base_models = [
#     ('lin_reg', LinearRegression()),
#     ('ridge', Ridge(alpha=1.0)),
#     ('lasso', Lasso(alpha=0.01)),
#     # ('elastic_net', ElasticNet(alpha=0.1, l1_ratio=0.5))
# ]

# # Define the meta-model
# meta_model = Ridge(alpha=1.0)

# # Stacking Regressor
# stacking_regressor = StackingRegressor(
#     estimators=base_models,
#     final_estimator=meta_model,
#     cv=5,
#     n_jobs=-1
# )

# # Full pipeline with stacking
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('stacking', stacking_regressor)
# ])

# # Hyperparameter tuning using GridSearchCV
# param_grid = {
#     'preprocessor__num__discretizer__n_bins': [5, 10],
#     'preprocessor__num__poly__degree': [2, 3],
#     'stacking__final_estimator__alpha': [0.1, 1.0, 10.0]
# }

# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
# grid_search.fit(X, y)

# # Best model evaluation
# best_model = grid_search.best_estimator_
# cv_scores = cross_val_score(best_model, X, y, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring='neg_mean_absolute_error')
# mean_cv_mae = -np.mean(cv_scores)
# print(f'Best Model Cross-validated Mean Absolute Error (MAE): {mean_cv_mae:.4f}')

# # Fit the best model on the entire training data
# best_model.fit(X, y)

# # Predict on the test set
# test_predictions = best_model.predict(X_test)

# # Ensure test_predictions length matches test_df
# if len(test_predictions) != len(test_df):
#     raise ValueError("The number of test predictions does not match the number of test data rows.")

# # Create submission file
# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'Age': test_predictions
# })
# submission.to_csv('submission.csv', index=False)

# print("Submission file created: submission.csv")


In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import Ridge, Lasso, HuberRegressor, TheilSenRegressor, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, FunctionTransformer, KBinsDiscretizer, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error


In [74]:

# # Load the datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Define the feature engineering function
# def feature_funk(df):
#     df['Length Bins'] = pd.qcut(df['Length'], q=4, labels=False)

#     sex = {'I': 1, 'F': 3, 'M': 2}
#     df['Sex'] = df['Sex'].replace(sex)
#     df = df[df['Sex'].isin([1, 2, 3])]
#     df['Sex'] = pd.to_numeric(df['Sex'])
#     return df

# # Apply the feature engineering function to both datasets
# train_df = feature_funk(train_df)
# test_df = feature_funk(test_df)

# # Define features and target
# X = train_df.drop(columns=['id', 'Age'])
# y = train_df['Age']
# X_test_original = test_df.drop(columns=['id'])


# # Define categorical and numerical columns
# categorical_cols = ['Sex']
# numerical_cols = [col for col in X.columns if col not in categorical_cols]

# # Define specific columns for discretization
# discretized_cols = ['Length', 'Weight']

# # Updated numerical preprocessing pipeline with RobustScaler and discretization for specific columns
# num_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('discretizer', KBinsDiscretizer(n_bins=5, encode='onehot', strategy='kmeans')),
#     ('poly', PolynomialFeatures(degree=3, include_bias=False)),  # You can lower the degree if necessary
#     ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
#     ('scaler', RobustScaler())
# ])

# # Updated categorical preprocessing pipeline
# cat_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
# ])

# # Combine all features
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', num_pipeline, [col for col in numerical_cols if col in discretized_cols]),
#         ('cat', cat_pipeline, categorical_cols),
#         ('age_group', OneHotEncoder(handle_unknown='ignore', drop='first'), ['Age_Group']),
#     ])

# # Define base models
# base_models = [
#     ('ridge', Ridge(alpha=10.0)),  # Increased alpha for more regularization
#     ('lasso', Lasso(alpha=0.1)),  # Adjust alpha
#     # ('ths', TheilSenRegressor(fit_intercept=True, random_state=42))
#     ('linear', LinearRegression())
# ]

# # Define the meta-model
# meta_model = LinearRegression()

# # Stacking Regressor
# stacking_regressor = StackingRegressor(
#     estimators=base_models,
#     final_estimator=meta_model,
#     cv=5,
#     n_jobs=-1
# )

# # Full pipeline with stacking
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('stacking', stacking_regressor)
# ])

# # Split the training data
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# # Cross-validation to evaluate performance
# cv = KFold(n_splits=5, shuffle=True, random_state=42)
# cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')
# mean_cv_mae = -np.mean(cv_scores)
# print(f'Stacking Cross-validated Mean Absolute Error (MAE): {mean_cv_mae:.4f}')

# # Fit the model on the entire training data
# pipeline.fit(X, y)

# # Predict on the validation set (used for MAE calculation here)
# # val_predictions = pipeline.predict(X_val)

# # # Calculate MAE for validation set
# # MAE = mean_absolute_error(y_val, val_predictions)
# # print(f"Validation MAE: {MAE:.4f}")

# # Fit the model on the entire training data and predict on the actual test set
# # pipeline.fit(X, y)
# test_predictions = pipeline.predict(X_test_original)

# # Create submission file
# submission5 = pd.DataFrame({
#     'id': test_df['id'],
#     'Age': test_predictions
# })
# submission5.to_csv('submissiontest.csv', index=False)

# print("Submission file created: submissiontest.csv")


In [75]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import KFold, cross_val_score, train_test_split
# from sklearn.linear_model import Ridge, Lasso, LinearRegression
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, FunctionTransformer, KBinsDiscretizer, RobustScaler
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import StackingRegressor
# from sklearn.metrics import mean_absolute_error

# # Load the datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Define the feature engineering function
# def feature_funk(df):
#     df['Length Bins'] = pd.qcut(df['Length'], q=4, labels=False)

#     sex = {'I': 1, 'F': 3, 'M': 2}
#     df['Sex'] = df['Sex'].replace(sex)
#     df = df[df['Sex'].isin([1, 2, 3])]
#     df['Sex'] = pd.to_numeric(df['Sex'])
#     return df

# # Apply the feature engineering function to both datasets
# train_df = feature_funk(train_df)
# test_df = feature_funk(test_df)

# # Define features and target
# X = train_df.drop(columns=['id', 'Age'])
# y = train_df['Age']
# X_test_original = test_df.drop(columns=['id'])

# # Define categorical and numerical columns
# categorical_cols = ['Sex']
# numerical_cols = [col for col in X.columns if col not in categorical_cols]

# # Updated numerical preprocessing pipeline with RobustScaler and discretization for specific columns
# num_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('discretizer', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')),  # Changed to 'ordinal' encoding
#     ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Lowered the degree to 2 for simplicity
#     ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
#     ('scaler', RobustScaler())
# ])

# # Updated categorical preprocessing pipeline
# cat_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
# ])

# # Combine all features
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', num_pipeline, numerical_cols),
#         ('cat', cat_pipeline, categorical_cols),
#     ])

# # Define base models
# base_models = [
#     ('ridge', Ridge(alpha=10.0)),  # Increased alpha for more regularization
#     ('lasso', Lasso(alpha=0.1)),  # Adjusted alpha
#     ('linear', LinearRegression()),
#     ("huber", HuberRegressor())
# ]

# # Define the meta-model
# meta_model = HuberRegressor()  # Using Ridge with adjusted alpha for regularization

# # Stacking Regressor
# stacking_regressor = StackingRegressor(
#     estimators=base_models,
#     final_estimator=meta_model,
#     cv=5,
#     n_jobs=-1
# )

# # Full pipeline with stacking
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('stacking', stacking_regressor)
# ])

# # Split the training data for validation
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Cross-validation to evaluate performance
# cv = KFold(n_splits=5, shuffle=True, random_state=42)
# cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')
# mean_cv_mae = -np.mean(cv_scores)
# print(f'Stacking Cross-validated Mean Absolute Error (MAE): {mean_cv_mae:.4f}')

# # Fit the model on the entire training data and predict on the validation set
# pipeline.fit(X_train, y_train)
# val_predictions = pipeline.predict(X_val)

# # Calculate MAE for validation set
# val_mae = mean_absolute_error(y_val, val_predictions)
# print(f"Validation MAE: {val_mae:.4f}")

# # Fit the model on the entire training data and predict on the actual test set
# pipeline.fit(X, y)
# test_predictions = pipeline.predict(X_test_original)

# # Create submission file
# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'Age': test_predictions
# })
# submission.to_csv('submissiontest1.csv', index=False)

# print("Submission file created: submissiontest1.csv")


In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import *
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV


# # Load the datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Define the target and features
# X = train_df.drop(columns=['id', 'Age'])
# y = train_df['Age']
# X_test = test_df.drop(columns=['id'])

# # Categorical and numerical columns
# categorical_cols = ['Sex']
# numerical_cols = X.columns.difference(categorical_cols)

# # Preprocessing pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', Pipeline([
#             ('imputer', SimpleImputer(strategy='mean')),
#             ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
#             ('scaler', StandardScaler())
#         ]), numerical_cols),
#         ('cat', Pipeline([
#             ('imputer', SimpleImputer(strategy='most_frequent')),
#             ('encoder', OneHotEncoder(drop='first'))
#         ]), categorical_cols)
#     ])

# # Model selection: you can experiment with HuberRegressor, GradientBoostingRegressor, etc.
# model = GradientBoostingRegressor(random_state=42)

# # Create the full pipeline with GradientBoostingRegressor
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('model', model)
# ])

# # Use cross-validation to assess the model's performance
# cv = KFold(n_splits=5, shuffle=True, random_state=42)
# cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_absolute_error')
# mean_cv_mae = -np.mean(cv_scores)
# print(f'Cross-validated Mean Absolute Error (MAE): {mean_cv_mae:.4f}')

# # Hyperparameter tuning using GridSearchCV
# param_grid = {
#     'model__n_estimators': [100, 200, 300],
#     'model__learning_rate': [0.01, 0.1, 0.2],
#     'model__max_depth': [3, 5, 7],
# }

# grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
# grid_search.fit(X, y)

# print(f'Best parameters found: {grid_search.best_params_}')
# print(f'Best cross-validated MAE: {-grid_search.best_score_:.4f}')

# # Train the model on the entire training set using the best parameters
# best_model = grid_search.best_estimator_
# best_model.fit(X, y)

# # Predict on the test set
# test_predictions = best_model.predict(X_test)

# # Prepare the submission file
# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'Age': test_predictions
# })

# # Save the submission file
# submission.to_csv('submission.csv', index=False)

# print("Submission file created: submission.csv")


In [77]:
# def feature_funk(df):
#     sex = {'I': 1, 'F': 3, 'M': 2}
#     df['Sex'] = df['Sex'].replace(sex)
#     df = df[df['Sex'].isin([1, 2, 3])]
#     df.loc[:, 'Sex'] = pd.to_numeric(df['Sex'], errors='coerce')

 
#     return df

# train_df = feature_funk(pd.read_csv('train.csv'))


# X = train_df.drop(columns=['id', 'Age'])
# y = train_df['Age']

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', Pipeline([
#             ('imputer', SimpleImputer(strategy='mean')),
#             ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
#             ('scaler', StandardScaler())
#         ]), X.columns),
#     ])
# cv = KFold(n_splits=5, shuffle=True, random_state=42)


# selector = RFECV(HuberRegressor(alpha=0.0001, max_iter=100000), step=1, cv=cv)

# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('selector', selector),
#     ('model', HuberRegressor(alpha=0.0001, max_iter=100000))

# ])


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')


# mean_cv_mae = -np.mean(cv_scores)
# pipeline.fit(X_train, y_train)

# y_pred = pipeline.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)

# print(f'CV RMAE: {np.mean(-cv_scores):.4f}')
# print(f'Test MAE: {mae:.4f}')
# print(f'Test MSE: {mse:.4f}')
# print(f'Test R^2 Score: {r2:.4f}')

In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import *
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import *


In [79]:
# Feature Engineering Function
def feature_funk(df):
    df['Surface Area'] = 2 * (df['Length'] * df['Diameter'] + df['Length'] * df['Height'] + df['Diameter'] * df['Height'])
    df['Vol'] = df['Length'] * df['Diameter'] * df['Height']
    df['Len_to_Dia_Ratio'] = df['Length'] / df['Diameter']
    df['Hgt_to_Wt_Ratio'] = df['Height'] / df['Weight']
    df['Sex'] = df['Sex'].map({'I': 1, 'F': 3, 'M': 2})
    return df

# Load Data and Apply Feature Engineering
train_df = feature_funk(pd.read_csv('train.csv'))
X = train_df.drop(columns=['id', 'Age'])
y = train_df['Age']

# Preprocessing Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', RobustScaler()),
            ('poly', PolynomialFeatures(degree=2))
        ]), X.columns)
    ])

# Simple Model Pipeline with Huber Regressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', HuberRegressor(alpha=0.0001, max_iter=100000))
])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Cross-Validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')

# Fit Model and Evaluate
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Output Results
print(f'CV MAE: {-np.mean(cv_scores):.4f}')
print(f'Test MAE: {mae:.4f}')
print(f'Test MSE: {mse:.4f}')
print(f'Test R^2 Score: {r2:.4f}')


KeyboardInterrupt: 